In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# An Introduction to Static Typing in TypeScript

TypeScript is a superset of JavaScript that adds a robust static type system to the language. Code written in TypeScript is compiled (transpiled) into JavaScript. During this process, the TypeScript compiler analyzes the code to ensure type safety.

This allows developers to catch errors at compile-time, long before the code runs in a browser or on a server. The type system is designed to be flexible and expressive, supporting:

* **Primitive types** (number, string, boolean)
* **Literal Types** (exact values as types)
* **Generic types** for reusable components
* **Union types** for values that can take multiple forms
* **Recursive types** for complex data structures

By utilizing type annotations, code becomes self-documenting, easier to refactor, and safer to execute.

## Finding Errors via Static Analysis

In a TypeScript environment, type checking happens before execution.

Consider the following example. We have an input string "5".
If we tell TypeScript that we expect a **numerical result** from our calculation, it will protect us from accidental string concatenation.

In [ ]:
const inputString: string = "5";

const result: number = inputString + 1;
result;

We can fix this by explicitly parsing the input string to a number. 

We pass `10` as the second argument (the **radix**) to ensure the string is interpreted as a decimal number, preventing accidental octal parsing of strings with leading zeros.

In [ ]:
const inputString: string = "5";

const result: number = parseInt(inputString, 10) + 1;
result;

## Type Annotations

The core of TypeScript is specifying types for variables, function parameters, and return values. 
* **Parameters:** Annotated by placing a colon `:` followed by the type after the parameter name.
* **Return Type:** Specified after the parameter list parentheses.

In [ ]:
function add(a: number, b: number): number {
    return a + b;
}

If we attempt to call this function with incompatible types (e.g., strings), the TypeScript compiler will raise an error. This prevents invalid data from flowing through the application.

In [ ]:
const myName = "Karl";

add("Hello ", myName); 

In [ ]:
add(10,5);

## Type Erasure

A fundamental concept in TypeScript is **Type Erasure**. Types exist **only** during development and compilation. 

Interfaces and type aliases are purely artifacts for the compiler. Once the code is compiled to JavaScript to run in Node.js or the browser, all type annotations are stripped away.

**Proof of Erasure:**
We cannot check against an interface at runtime using `instanceof` or `typeof`, because the interface simply does not exist in the JavaScript output.

In [ ]:
interface User {
    id: number;
    name: string;
}

const u = { id: 1, name: "Alice" };

// Compile-time check works:
const validUser: User = u; 

// Runtime-Check FAILS
if (u instanceof User) { 
    console.log("Is User");
}

## Built-in Types

TypeScript supports all standard JavaScript primitives (`number`, `string`, `boolean`) and complex structures like Arrays (`number[]` or `Array<number>`).

The function `average` below demonstrates working with arrays of numbers.

In [ ]:
function average(numbers: number[]): number {
    if (numbers.length === 0) return 0;
    const total = numbers.reduce((acc, curr) => acc + curr, 0);
    return total / numbers.length;
}

In [ ]:
average([1, 2, 3, 4])

## Custom Types and Classes

We can define custom data structures using `class` or `interface`. In a TypeScript class, properties must be declared and initialized. The `public` keyword can be used to automatically define and assign properties in the constructor.

In [ ]:
class Person {
    name: string;

    constructor(name: string) {
        this.name = name;
    }

    greet(): string {
        return `Hello, ${this.name}!`;
    }
}

When a function performs an action but does not return a value, its return type is `void`.

In [ ]:
function salve(p: Person): void {
    console.log(p.greet());
}

In [ ]:
const jc = new Person('Julius Caesar');
salve(jc);

## Union Types and Type Narrowing

Sometimes a value can be of more than one type. The **Union Type** operator `|` allows us to define a variable that can hold, for example, either a `string` OR an object.

To work with these values safely, we use **Type Narrowing** (e.g., checking `typeof`) to determine the specific type at runtime.

In [ ]:
type NameDict = { given: string; family: string };

function greetName(name: string | NameDict): string {
    if (typeof name === 'string') {
        return 'Hi ' + name + '!';
    } 
    
    return `Bienvenido, Señor ${name.given} ${name.family}.`;
}

In [ ]:
greetName("Alice")

In [ ]:
greetName({ given: 'Esteban', family: 'Ramirez' })

## Literal Types

In TypeScript, specific strings or numbers can be treated as types themselves. This is extremely useful for defining finite states or specific configurations.

Instead of just saying a variable is a `string`, we can say it is specifically `"north"`, `"south"`, `"east"`, or `"west"`.

In [ ]:
type Direction = "North" | "South" | "East" | "West";

function move(dir: Direction) {
    console.log("Moving " + dir);
}

move("North");

In [ ]:
move("Up");

## Escape Hatches (`any` vs `unknown`)

Sometimes strictness gets in the way.

### 1. The `any` type (The "Off Switch")
The `any` type disables type checking. It allows you to access any property. It effectively reverts TypeScript back to standard JavaScript behavior for that variable. **Never ever use this.**

### 2. The `unknown` type (The Safe Alternative)
`unknown` is the type-safe counterpart. It represents "anything", but TypeScript won't let you use it until you prove what it is (narrowing).

In [ ]:
let looseValue: any = 4;
looseValue.toUpperCase(); // Crashes at Runtime

In [ ]:
let safeValue: unknown = "hello";

In [ ]:
safeValue.toUpperCase(); // Error at Compile-Time

In [ ]:
if (typeof safeValue === 'string') {
    console.log(safeValue.toUpperCase());
}

## Overriding the Compiler: `as` and `!`

While strict typing is safer, there are situations where we need to override the compiler's decisions. TypeScript provides mechanisms for this, but they should be used with **extreme caution**.

Using these operators effectively **turns off the type checker** for that specific expression. By using them, you take full responsibility for type safety. If you are wrong, the compiler will not warn you, and your program may crash at runtime.

### 1. Type Assertions (`as`)

The `as` keyword tells the compiler to treat a value as a specific type, regardless of what the compiler inferred.

* **When to use:** Only when you have external knowledge about a type that TypeScript cannot know (e.g., data coming from a specific API endpoint).
* **The Risk:** You are essentially "lying" to the compiler. If the actual value does not match your assertion, strict typing provides zero protection.

In [ ]:
let someValue: unknown = "hello world";

let numberLength = (someValue as number).toFixed(2); // Crashes at Runtime

### 2. The Non-Null Assertion Operator (`!`)

The exclamation mark `!` after a variable removes `null` and `undefined` from its type.

* **When to use:** When you are absolutely certain a value exists, even though the type definition says it might be missing.
* **The Risk:** If the value *is* actually `null` or `undefined` at runtime, accessing properties on it will cause the program to crash immediately.

In [ ]:
function findUser(id: number): string | undefined {
    return undefined;
}

const user = findUser(42)!;
user.toUpperCase(); // Crashes at Runtime

## Generics

Generics allow us to write reusable code that works with a variety of types while maintaining strict type safety. Think of them as **"Variables for Types"**.

We use type variables (conventionally `<T>`, `<S>`, etc.) to represent these types.
TypeScript can usually **infer** the generic type from the arguments, but we can also specify it **explicitly**.

In [ ]:
function swap<A, B>(pair: [A, B]): [B, A] {
    const [x, y] = pair;
    return [y, x];
}

### 1. Inference: TypeScript sees number and string

In [ ]:
const swapped = swap([1, 'a']);
swapped;

In [ ]:
const swapped2 = swap<boolean, number>([true, 42]);
swapped2;

## Advanced: The `Structural` Interface

In JavaScript, objects are compared by **reference**, not by value. 
`{a: 1} === {a: 1}` is `false`.

For our Formal Languages project (FSMs), we need to store complex states (sets of numbers, sets of sets) in data structures. To do this efficiently, we use a custom library `recursive-set` which relies on **Structural Equality**.

Objects must implement the `Structural` interface to be stored in these special Sets and Maps.

In [ ]:
interface Structural {
    /** A stable hash code for the object. */
    readonly hashCode: number;

    /** Checks structural equality with another object. */
    equals(other: unknown): boolean;

    /** Returns a deterministic string representation. */
    toString(): string;
}

In [ ]:
class Point implements Structural {
    constructor(public x: number, public y: number) {}

    get hashCode(): number {
        return this.x + this.y * 31;
    }

    equals(other: unknown): boolean {
        return other instanceof Point && 
               this.x === other.x && 
               this.y === other.y;
    }

    toString(): string {
        return `P(${this.x},${this.y})`;
    }
}

## Advanced: Typed Tuples

To handle pairs like `(State, InputChar)` efficiently, we use a custom `Tuple` class (from `recursive-set`). 

A key feature of this class is its `get` method, which uses TypeScript's **advanced lookup types** to ensure type safety. 
* It knows exactly that index `0` is a `string`.
* It knows that index `1` is a `number`.

**Note:** If we try to access an index out of bounds (like `2`), a strict compiler would reject this code. However, at runtime (as seen below), JavaScript simply returns `undefined`.

In [ ]:
import { Tuple } from "recursive-set";

const t = new Tuple("S0", 42); 

// TypeScript knows index 0 is a string
const label = t.get(0);
console.log(`Index 0 is type: ${typeof label} ("${label}")`);

// TypeScript knows index 1 is a number
const val = t.get(1);
console.log(`Index 1 is type: ${typeof val} (${val})`);

// Accessing out of bounds (Index 2)
// Ideally a compile error, but at runtime it is undefined:
const error = t.get(2); 

console.log(`Index 2 is: ${error}`);